In [1]:
!pip install simpy
import simpy
import random

In [2]:
RANDOM_SEED = 42
NUM_RUNWAY = 1    # Number of runway in the airport
TAKEOFFTIME = 5   # Minutes airplane takes to take off
LANDTIME = 6      # Minutes airplane takes to land
DEPARTURE_INTER = 10  # Create departure airplane every 10 minutes
LAND_INTER = 10      # Create land airplane every 10 minutes
AIRPLANE_CAPACITY = 200 # airplane capacity is 200
TIME_ON_GROUND = 45 #Arrival aircraft spends 45 minutes at the terminal for de-boarding and boarding (this doesn't include departure schedule)
AIRPLANES_AT_TERMINAL = 5 #Total aircrafts present at the termianl at the start of the simulation
AIRPLANES_SCHEDULED_TO_LAND_TODAY = 10 #Total aircrafts expected to arrive and land today at the airport


# NEW_DEPARTURES = AIRPLANES_AT_TERMINAL - AIRPLANES_SCHEDULED_TO_LAND_TODAY #This is to indicate the airplanes which will depart the airspace after arriving today

In [4]:
class Runnway(object):
    def __init__(self, env, num_runway):
        self.env = env
        self.runway = simpy.Resource(env, num_runway)

    def occupy(self, occupy_time): #function indicating whether runway is occupied or idle
        yield self.env.timeout(occupy_time)

In [5]:
class Airplane(object):
    def __init__(self, env, name, rw, takeoff_time, land_time):
        self.env = env
        self.rw = rw
        self.name = name
        self.takeoff_time = takeoff_time
        self.land_time = land_time
        self.passager = 180 #90% capacity
        self.late = False #check if the airplane is late (default is False)
        self.status = False #Status of Takeoff
        
    def takeOff(self):
        self.status = True
        print('%s ready to take off at %.2f.' % (self.name, env.now))
        with self.rw.runway.request() as request:
            yield request
            print('%s enters the runway at %.2f.' % (self.name, env.now))
            yield env.process(self.rw.occupy(self.takeoff_time))
            print('%s leaves the Airport at %.2f.' % (self.name, env.now))
        
    def land(self):
        print('%s ready to land %.2f.' % (self.name, env.now))
        with self.rw.runway.request() as request:
            yield request
            print('%s enters the runway at %.2f.' % (self.name, env.now))
            yield env.process(self.rw.occupy(self.land_time))
            print('%s leaves the runway at %.2f. and enters the terminal' % (self.name, env.now))
            self.status = True
        
        yield env.timeout(TIME_ON_GROUND)
        with self.rw.runway.request() as request:
            yield request
            print('%s leaves the terminal and enters the runway at %.2f.' % (self.name, env.now))
            yield env.process(self.rw.occupy(self.takeoff_time))
            print('%s leaves the Airport at %.2f.' % (self.name, env.now))


    def schedule(self, time):
        yield env.timeout(time)
        # Skip if the plane has already taken off
        if self.status:
            pass
        # Only those with more than 180 boardings are allowed to take off
        if self.passager >= 180:
            # ready to take off 
            env.process(self.takeOff())
        else:
            print('%s late at %.2f.' % (self.name, env.now))
            self.late = True
            
    def boarding(self, passager_num):
        if self.passager < 200:
            self.passager += passager_num


In [6]:
# Schedule the flight departure time
class Departure_Table(object):
    def __init__(self, env, num_airplane, time_interval, runway):
        self.num_airplane = num_airplane
        self.time_interval = time_interval
        self.runway = runway
        self.airplanes = []
    
    def schedule(self):
        for i in range(self.num_airplane):
            airplane = Airplane(env, 'Depart_airplane %d' % i, self.runway, TAKEOFFTIME, LANDTIME)
            self.airplanes.append(airplane)
            env.process(airplane.schedule( i * self.time_interval))


In [7]:
# Schedule the flight land time
class Land_Table(object):
    def __init__(self, env, num_airplane, time_interval, runway):
        self.num_airplane = num_airplane
        self.time_interval = time_interval
        self.runway = runway
        self.airplanes = []
    
    def schedule(self):
        for i in range(self.num_airplane):
            yield env.timeout(self.time_interval)
            airplane = Airplane(env, 'Land_airplane %d' % i, self.runway, TAKEOFFTIME, LANDTIME)
            self.airplanes.append(airplane)
            env.process(airplane.land())


In [8]:
def ATF(env):
    runway = Runnway(env, NUM_RUNWAY) #Creating runway object
    
    departure_Table = Departure_Table(env, AIRPLANES_AT_TERMINAL, DEPARTURE_INTER, runway)
    departure_Table.schedule()
    
    land_Table = Land_Table(env, AIRPLANES_SCHEDULED_TO_LAND_TODAY, LAND_INTER, runway)
    env.process(land_Table.schedule())
    
    while True:
        # Every 1 time step, check whether the flights that are not on time can take off
        yield env.timeout(1)
        for plane in departure_Table.airplanes:
            if plane.status == False:
                if plane.late and plane.passager > 180:
                    env.process(plane.takeOff())
        
        
# Setup and start the simulation
print('Airport Runway Traffic Simulation')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(ATF(env))

# Execute!
env.run(1000)

Airport Runway Traffic Simulation
Depart_airplane 0 ready to take off at 0.00.
Depart_airplane 0 enters the runway at 0.00.
Depart_airplane 0 leaves the Airport at 5.00.
Depart_airplane 1 ready to take off at 10.00.
Land_airplane 0 ready to land 10.00.
Depart_airplane 1 enters the runway at 10.00.
Depart_airplane 1 leaves the Airport at 15.00.
Land_airplane 0 enters the runway at 15.00.
Depart_airplane 2 ready to take off at 20.00.
Land_airplane 1 ready to land 20.00.
Land_airplane 0 leaves the runway at 21.00. and enters the terminal
Depart_airplane 2 enters the runway at 21.00.
Depart_airplane 2 leaves the Airport at 26.00.
Land_airplane 1 enters the runway at 26.00.
Depart_airplane 3 ready to take off at 30.00.
Land_airplane 2 ready to land 30.00.
Land_airplane 1 leaves the runway at 32.00. and enters the terminal
Depart_airplane 3 enters the runway at 32.00.
Depart_airplane 3 leaves the Airport at 37.00.
Land_airplane 2 enters the runway at 37.00.
Depart_airplane 4 ready to take of